In [1]:
import os
import pandas as pd

import ssl
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

import ywc_email_template

In [2]:
letters = pd.read_excel("camper-acceptance-letters/รายชื่อหนังสือรับรอง camper.xlsx")
letters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   twa_ref          26 non-null     object
 1   camper_title     26 non-null     object
 2   camper_fullname  26 non-null     object
 3   camper_major     26 non-null     object
 4   camper_desc      26 non-null     object
 5   recipient        26 non-null     object
 6   to_email         26 non-null     object
 7   cc_emails        2 non-null      object
dtypes: object(8)
memory usage: 1.8+ KB


In [3]:
letters = letters[letters["twa_ref"] == "สผวท. 066/66"]

In [4]:
letters["twa_ref"].apply(lambda x: "สผวท. " + x.split(" ")[1].replace("/","-") + ".pdf")

25    สผวท. 066-66.pdf
Name: twa_ref, dtype: object

In [5]:
letters["filename"] = letters["twa_ref"].apply(
    lambda x: "สผวท. " + x.split(" ")[1].replace("/", "-") + ".pdf"
)

In [6]:
with open("camper-acceptance-letters/เอกสารโครงการ Young Webmaster Camp ครั้งที่ 19.pdf", "rb") as file:
    ywc19_pdf_file = MIMEApplication(file.read())
    ywc19_pdf_file.add_header(
        "Content-Disposition",
        "attachment",
        filename="เอกสารโครงการ Young Webmaster Camp ครั้งที่ 19.pdf",
    )

In [7]:
import getpass

sender_email = "ywc@webmaster.or.th"
sender_password = getpass.getpass("Password: ")

In [8]:
context = ssl.create_default_context()

with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
    smtp.login(sender_email, sender_password)

    for letter in list(letters.itertuples()):
        msg = MIMEMultipart()

        msg["From"] = sender_email
        msg["To"] = letter.to_email

        recipients = list()
        recipients.append(letter.to_email)
        if not pd.isnull(letter.cc_emails):
            recipients += letter.cc_emails.split(",")
            msg["Cc"] = letter.cc_emails

        msg["Subject"] = (
            "[YWC19] ส่งเอกสารรับรองการคัดเลือกเข้าร่วมโครงการ - "
            + letter.camper_fullname
        )

        with open(f"camper-acceptance-letters/letters/{letter.filename}", "rb") as file:
            letter_pdf_file = MIMEApplication(file.read())
            letter_pdf_file.add_header(
                "Content-Disposition",
                "attachment",
                filename=f"หนังสือรับรองการคัดเลือกเข้าร่วมโครงการ YWC19 - {letter.camper_fullname}.pdf",
            )
            
        
        msgText = MIMEText(
            ywc_email_template.build_camper_acceptance_letter_email(
                camper_title=letter.camper_title,
                camper_fullname=letter.camper_fullname,
                camper_major=letter.camper_major,
            ),
            "html",
        )
        msg.attach(msgText)
        msg.attach(letter_pdf_file)
        msg.attach(ywc19_pdf_file)
        
        smtp.sendmail(
            sender_email,
            recipients,
            msg.as_string(),
        )
        print(f"Successfully sent {letter.filename}")

Successfully sent สผวท. 066-66.pdf
